In [134]:
import IPython, ipywidgets, traitlets, requests, pandas, jinja2
from bokeh.sampledata import iris
%reload_ext yamlmagic
%reload_ext autoreload
%autoreload 2

In [145]:
access_token = ipywidgets.Text('asdfasdf')
access_token

In [143]:
class Projects( traitlets.HasTraits ):
    current = traitlets.List([])
    def __init__( self, *args, **kwargs ):
        self.catalog = {}   
        self.session = requests.Session()
        
    @traitlets.observe('current')
    def _update_projects(self, change, *args, **kwargs):
        for project_or_url in change['new']:
            self._github_api( project_or_url )
                
    def _github_api( self, project_or_url, params = {} ):
        if not project_or_url.startswith('http'):
            user, repo = project_or_url.split('/')
            url = None
        else:
            url = project_or_url
        if not url:
            url = 'https://api.github.com/repos/' + user + '/' + repo
                
        request = requests.Request('GET', url=url, params=params.copy(), headers={'User-Agent': 'tonyfast'} )
        url = request.prepare().url
        if not url in self.catalog.keys():            
            request.params['access_token'] = globals()['access_token'].value
            self.catalog[url] = {
                'requests': self.session.send(request.prepare()),
                'time': str(pandas.datetime.now()),
                'params': params.copy(),
            } 
        return url
    
    def show( self ):
        pass
    
    def more( self, url, params = {}, paginate=False ):
        if paginate:
            last_page = [v['params']['page'] for k, v in self.catalog.items() if k.startswith(url) and 'page' in v['params']]
            params['page'] = max(last_page) + 1 if last_page else 1
        print(params)
        return self.catalog[ self._github_api( url, params = params ) ]
    
    def update( self, project_name ):
        tmp = list( self.current )
        tmp.append( project_name )
        self.current = tmp
        
projects = Projects()

In [137]:
projects.current = ['jupyter/notebook', 'bokeh/bokeh']

In [138]:
projects.catalog['https://api.github.com/repos/jupyter/notebook']['requests']

<Response [200]>

In [139]:
iss = projects.more('https://api.github.com/repos/jupyter/notebook/issues', paginate=True)

{'page': 1}


In [140]:
iss = projects.more('https://api.github.com/repos/jupyter/notebook/issues', paginate=True)

{'page': 2}
